<div >
<img src = "banner.jpg" />
</div>

# The Bootstrap

<div >
<img src = "pelos.jpg" />
</div>


## Introduction


The bootstrap is a widely applicable and extremely powerful statistical tool that can be used to quantify the uncertainty associated with a given estimator or statistical learning method. As a simple example, the bootstrap can be used to estimate the standard errors of the coefficients from a linear regression fit. In the specific case of linear regression, this is not particularly useful, since standard statistical software outputs such standard errors automatically. However, the power of the bootstrap lies in the fact that it can be easily applied to a wide range of statistical learning methods, including some for which a measure of variability is otherwise difficult to obtain and is not automatically output by statistical software.

Let's ilustrate with an example. Suppose we have the following data

In [ ]:
    gas<-read.csv("https://raw.githubusercontent.com/ignaciomsarmiento/datasets/main/gas.csv",header=T)
    head(gas)

## Case 1

Where all the variables are in logs, and would like to estimate an equation like this: 

$$\ln{Quantity}_{t}= \alpha + \beta_1 \ln{Price}_{t} +\beta_2 \ln{Income}_{t} +u_{t}$$

In [ ]:
require("pacman")
p_load("tidyverse","stargazer")
mod1<- lm(consumption~price+income,gas)
stargazer(mod1,type="text")

In [ ]:
set.seed(112)

R<-1000 # Number of Repetions()

eta_mod1<-rep(0,R)


In [ ]:
for (i in 1:R){
        
      db_sample<- sample_frac(gas,size=1,replace=TRUE)
        
      f<-lm(consumption~price+income,db_sample)

      coefs<-f$coefficients
    
      eta_mod1[i]<-coefs[2]
    }

In [ ]:
plot(hist(eta_mod1))

In [ ]:
mean(eta_mod1)

In [ ]:
quantile(eta_mod1,c(0.025,0.975))

In [ ]:
sqrt(var(eta_mod1))

In [ ]:
p_load("boot")



eta.fn<-function(data,index){
  coef(lm(consumption~price+income, data = data, subset = index))
}

boot(gas, eta.fn, R = 1000)

## Case 2

$$\ln{Quantity}_{t}= \alpha + \beta_1 \ln{Price}_{t}  + \beta_2 \ln{Price}^2_{t} +\beta_3 \ln{Income}_{t}+ \beta_4 \ln{Price}_{t}\times \ln{Income}_{t} +u_{t}$$




first you need first to generate the quadratic and other terms as follows: 

In [ ]:
  gas<- gas %>% mutate(price2=price^2, 
                         price_income=price*income )

then regress: 

In [ ]:
mod2<-lm(consumption~price+price2+income+price_income,gas)
stargazer(mod1,mod2,type="text")

 To do that you will need to: 
 
i) Obtain the coefficients of regression: 

In [ ]:
    coefs<-mod2$coef
    coefs 

ii) Extract the coefficients to scalars: 

In [ ]:
    b0<-coefs[1] 
    b1<-coefs[2]
    b2<-coefs[3] 
    b3<-coefs[4]
    b4<-coefs[5] 

iii) Calculate the elasticity at the sample mean:

In [ ]:
price_bar<-mean(gas$price)
income_bar<-mean(gas$income)


elastpt<-b1+2*b2*price_bar+b4*income_bar
    
elastpt

How do we calculate the standard errors?


In [ ]:

eta_mod2.fn<-function(data,index,
                      price_bar=mean(gas$price),
                      income_bar=mean(gas$income)){
      
      f<-lm(consumption~price+price2+income+price_income,data, subset = index)
        coefs<-f$coefficients
        
        b1<-coefs[2]
        b2<-coefs[3] 
        b4<-coefs[5] 
    
      elastpt<-b1+2*b2*price_bar+b4*income_bar
    
    
      return(elastpt)
    }

eta_mod2.fn(gas,1:nrow(gas))

    

In [ ]:
results <- boot(data=gas, eta_mod2.fn,R=1000)
results